In [33]:
import networkx as nx
import numpy as np
import time
import random
random.seed(10)
from random import random as rnd

NUMBER_OF_NODES=200
NUMBER_OF_EDGES=1000

In [34]:
time1=time.clock()
testingGNM = nx.gnm_random_graph(NUMBER_OF_NODES,NUMBER_OF_EDGES,seed=24,directed=False)
GraphList = []
for x,(i,j) in enumerate(testingGNM.edges()):
    GraphList.append([i,j,int(99*rnd())+1])
time2=time.clock()
print("Time to generate graph: ",time2-time1)

Time to generate graph:  0.0038072086252896042


In [35]:
print(nx.is_connected(testingGNM))

True


In [36]:
del testingGNM

In [37]:
# print(GraphList)

In [38]:
# -------------------------- BORUVKA MST
# -------------------------- Code by Neelam Yadav https://www.geeksforgeeks.org/boruvkas-algorithm-greedy-algo-9/
#  Boruvka's algorithm to find Minimum Spanning 
# Tree of a given connected, undirected and weighted graph 
  
from collections import defaultdict 
  
#Class to represent a graph 
class Graph: 
  
    def __init__(self,vertices): 
        self.V= vertices #No. of vertices 
        self.graph = [] # default dictionary to store graph 

          
   
    # function to add an edge to graph 
    def addEdge(self,u,v,w): 
        self.graph.append([u,v,w]) 
  
    # A utility function to find set of an element i 
    # (uses path compression technique) 
    def find(self, parent, i): 
        if parent[i] == i: 
            return i 
        return self.find(parent, parent[i]) 
  
    # A function that does union of two sets of x and y 
    # (uses union by rank) 
    def union(self, parent, rank, x, y): 
        xroot = self.find(parent, x) 
        yroot = self.find(parent, y) 
  
        # Attach smaller rank tree under root of high rank tree 
        # (Union by Rank) 
        if rank[xroot] < rank[yroot]: 
            parent[xroot] = yroot 
        elif rank[xroot] > rank[yroot]: 
            parent[yroot] = xroot 
        #If ranks are same, then make one as root and increment 
        # its rank by one 
        else : 
            parent[yroot] = xroot 
            rank[xroot] += 1
  
    # The main function to construct MST using Kruskal's algorithm 
    def boruvkaMST(self): 
        MSTlines=[]
        parent = []; rank = [];  
  
        # An array to store index of the cheapest edge of 
        # subset. It store [u,v,w] for each component 
        cheapest =[] 
  
        # Initially there are V different trees. 
        # Finally there will be one tree that will be MST 
        numTrees = self.V 
        MSTweight = 0
  
        # Create V subsets with single elements 
        for node in range(self.V): 
            parent.append(node) 
            rank.append(0) 
            cheapest =[-1] * self.V 
      
        # Keep combining components (or sets) until all 
        # compnentes are not combined into single MST 
  
        while numTrees > 1: 
  
            # Traverse through all edges and update 
               # cheapest of every component 
            timeBEFOREFINDMIN=time.clock()
            for i in range(len(self.graph)): 
  
                # Find components (or sets) of two corners 
                # of current edge 
                u,v,w =  self.graph[i] 
                set1 = self.find(parent, u) 
                set2 = self.find(parent ,v) 
  
                # If two corners of current edge belong to 
                # same set, ignore current edge. Else check if  
                # current edge is closer to previous 
                # cheapest edges of set1 and set2 
                if set1 != set2:      
                      
                    if cheapest[set1] == -1 or cheapest[set1][2] > w : 
                        cheapest[set1] = [u,v,w]  
  
                    if cheapest[set2] == -1 or cheapest[set2][2] > w : 
                        cheapest[set2] = [u,v,w] 
            timeAFTERFINDMIN=time.clock()
            print("Time to find min edges: ",timeAFTERFINDMIN-timeBEFOREFINDMIN)
            # Consider the above picked cheapest edges and add them 
            # to MST 
            for node in range(self.V): 
                counter=1
                #Check if cheapest for current set exists 
                if cheapest[node] != -1: 
                    u,v,w = cheapest[node] 
                    set1 = self.find(parent, u) 
                    set2 = self.find(parent ,v) 
  
                    if set1 != set2 : 
                        MSTweight += w 
                        self.union(parent, rank, set1, set2) 
#                         print ("%d: Edge %d-%d with weight %d included in MST" % (counter,u,v,w)) 
                        counter+=1
#                         MSTlines.append([tuple(self.X[u]),tuple(self.X[v])])
                        numTrees = numTrees - 1

            #reset cheapest array 
            cheapest =[-1] * self.V 
  
              
        print ("Weight of MST is %d" % MSTweight)
#         return MSTlines

In [39]:
FullGraph = Graph(NUMBER_OF_NODES)
for i in GraphList:
    FullGraph.addEdge(i[0],i[1],i[2])

In [40]:
time1 = time.clock()
FullGraph.boruvkaMST()
time2 = time.clock()
print('MST with full graph took %f seconds' % (time2-time1))
del FullGraph

Time to find min edges:  0.0005511407873655116
Time to find min edges:  0.0007426119631190886
Time to find min edges:  0.0009104577818561665
Weight of MST is 2768
MST with full graph took 0.003345 seconds


In [41]:
# Author: Nitin B. Tangellamudi
import numpy as np
from numpy import linalg as la
from pycuda import driver, compiler, gpuarray, tools
import string

# Initialize Device
import pycuda.autoinit

# Boruvka's algorithm to find Minimum Spanning 
# Tree of a given connected, undirected and weighted graph 
  
from collections import defaultdict 
  
    
def make_numpy(arr):
    a = np.empty(len(arr),dtype=np.int32)
    for i in range(len(arr)):
        a[i]=arr[i]
    return a

def make_numpy2(arr):
    a = np.empty((len(arr),len(arr[0])),dtype=np.int32)
    for i in range(len(arr)):
        a[i]=arr[i]
    return a

#Class to represent a graph 
class GraphParallel: 
  
    def __init__(self,vertices): 
        self.V= vertices #No. of vertices 
        self.graph = [] # default dictionary to store graph 
          
   
    # function to add an edge to graph 
    def addEdge(self,u,v,w): 
        self.graph.append([u,v,w]) 
  
    # A utility function to find set of an element i 
    # (uses path compression technique) 
    def find(self, parent, i): 
        if parent[i] == i: 
            return i 
        return self.find(parent, parent[i]) 
  
    # A function that does union of two sets of x and y 
    # (uses union by rank) 
    def union(self, parent, rank, x, y): 
        xroot = self.find(parent, x) 
        yroot = self.find(parent, y) 
  
        # Attach smaller rank tree under root of high rank tree 
        # (Union by Rank) 
        if rank[xroot] < rank[yroot]: 
            parent[xroot] = yroot 
        elif rank[xroot] > rank[yroot]: 
            parent[yroot] = xroot 
        #If ranks are same, then make one as root and increment 
        # its rank by one 
        else : 
            parent[yroot] = xroot 
            rank[xroot] += 1
     
    def makeInputArray(self):
        D = [[] for i in range(self.V)]
        for edge in self.graph:
            D[edge[0]].extend(edge)
            D[edge[1]].extend(edge)
        flattened_D = [y for x in D for y in x]
        helperD1 = np.array([int(len(x)/3) for x in D],dtype=np.uint32)
        helperD2 = np.cumsum(helperD1)
        
        return np.asarray(flattened_D,dtype=np.uint32),helperD1,helperD2
                
        
    

    
    
    # The main function to construct MST using Kruskal's algorithm 
    def boruvkaMST(self,cArray,cHelper1,cHelper2):
                    
        kernel = ("""
            #include <stdio.h>
            __device__ int find(int *parent, int i){
                if(parent[i]==i){
                    //printf("here 1 ");
                    return i;
                    }
                return find(parent,parent[i]);
            }
            
            __global__ void findmin(int *cArray, int *cHelper1,int *cHelper2, int graphLen, int numNodes,int numEdges, int *graph, int *parent, int *cheapest)
            {
                __syncthreads();
                const unsigned int i = blockDim.x*blockIdx.x + threadIdx.x;
                const unsigned int row = blockDim.y*blockIdx.y + threadIdx.y;
           
                if(i<=numNodes){
                
                    int edgeSrc=-1;
                    int edgeTerm = -1;
                    int edgeW = -1;
        
                    for(int edgeCtr = 0; edgeCtr< cHelper1[i];edgeCtr++){
                        //let us redefine the edges we are considering
                        
                        if(i==0){
                            edgeSrc = cArray[3*edgeCtr];
                            edgeTerm = cArray[3*edgeCtr+1];
                            edgeW = cArray[3*edgeCtr+2];
                            int set1 = find(parent,edgeSrc);
                            int set2 = find(parent,edgeTerm);

                            if(set1!=set2){                    
                                if ( (cheapest[set1] == -1) || (cArray[cheapest[set1]+2] > edgeW) ){                         
                                    cheapest[set1] = 3*edgeCtr;

                            }
                                if( (cheapest[set2] == -1) || (cArray[cheapest[set2]+2] > edgeW) ){
                                    cheapest[set2] = 3*edgeCtr;
                            }
                        }
                        }
                        else{
                            edgeSrc = cArray[3*cHelper2[i-1]+3*edgeCtr];
                            edgeTerm = cArray[3*cHelper2[i-1]+3*edgeCtr+1];
                            edgeW = cArray[3*cHelper2[i-1]+3*edgeCtr+2];
                            int set1 = find(parent,edgeSrc);
                            int set2 = find(parent,edgeTerm);

                            if(set1!=set2){                    
                                if ( (cheapest[set1] == -1) || (cArray[cheapest[set1]+2] > edgeW) ){                         
                                    cheapest[set1] = 3*cHelper2[i-1]+3*edgeCtr;

                            }
                                if( (cheapest[set2] == -1) || (cArray[cheapest[set2]+2] > edgeW) ){
                                    cheapest[set2] = 3*cHelper2[i-1]+3*edgeCtr;
                            }
                        }                            
                        }
                        
                    }

                
                }

            }



        """)
       

        parent = []; rank = [];  
  
        # An array to store index of the cheapest edge of 
        # subset. It store [u,v,w] for each component 
        cheapest =[] 
  
        # Initially there are V different trees. 
        # Finally there will be one tree that will be MST 
#         numTreesO = np.zeros(1,dtype=np.int32)
#         numTreesO[0] = self.V
        numTrees = self.V
        numNodes = np.uint32(self.V)
        MSTweight = 0
          
        # Create V subsets with single elements 
        for node in range(self.V): 
            parent.append(node)
            rank.append(0) 
            cheapest =[-1] * self.V 

        NUM_NODESSTR=str(self.V)
        
        mod = compiler.SourceModule(kernel)
        
        findMin = mod.get_function("findmin")
        findMin.prepare([np.intp,np.intp,np.intp,np.int32,np.int32,np.int32,np.intp,np.intp,np.intp])
        
        numEdges  = np.int32(len(self.graph))
        nodel = np.int32(self.V)
        
        parent2 = make_numpy(parent)

        graph2 = np.squeeze(np.reshape(make_numpy2(self.graph),(len(self.graph)*3,-1)))
        graphLen = np.uint32(len(graph2))
#         rank2 = make_numpy(rank)
        cheapest2 = make_numpy(cheapest)
        parent_gpu = gpuarray.to_gpu(parent2.astype(np.int32))
        graph_gpu = gpuarray.to_gpu(graph2.astype(np.int32))
#         print(graph_gpu.get())
#         rank_gpu = gpuarray.to_gpu(rank2.astype(np.int32))
        cheapest_gpu = gpuarray.to_gpu(cheapest2.astype(np.int32))
        cArray_gpu = gpuarray.to_gpu(cArray.astype(np.int32))
        cHelper1_gpu = gpuarray.to_gpu(cHelper1.astype(np.int32))
        cHelper2_gpu = gpuarray.to_gpu(cHelper2.astype(np.int32))
#         print("Cheapest_GPU before launch: ",cheapest_gpu.get())
        
        
        if(len(self.graph)>992):
            block = (1024,1,1)
            block2 = (1024,1,1)
            grid=(int(len(self.graph)/1024)+1,1)
        else:
            block = (32*int(len(self.graph)/32)+32,1,1)
            block2 = (32*int(len(self.graph)/32)+32,1,1)
            grid = (1,1)
        print("Number of Nodes: ",self.V)
        print("Number of Edges: ",len(self.graph))
        print("Size of block: ",block)
        print("Size of grid: ",grid)
#         print(graph_gpu.get())
        while numTrees > 1:
            timeBEFOREFINDMIN = time.clock()
            findMin.prepared_call(grid,block,cArray_gpu.gpudata,cHelper1_gpu.gpudata,cHelper2_gpu.gpudata,graphLen,numNodes,numEdges,graph_gpu.gpudata,parent_gpu.gpudata,cheapest_gpu.gpudata)
            timeAFTERFINDMIN=time.clock()
            print("Time CUDA code executes: ",timeAFTERFINDMIN-timeBEFOREFINDMIN)
        # Traverse through all edges and update 
#             print('before')
            
            cheapest = cheapest_gpu.get()
#             print("CHEAPEST: ",cheapest)
            parent = parent_gpu.get()
#             print("This is the cheapest array: ",cheapest)
            for node in range(self.V): 
#                 print(cheapest)
                #Check if cheapest for current set exists 
                if cheapest[node] != -1: 
                    u = cArray[cheapest[node]]
                    v = cArray[cheapest[node]+1]
                    w = cArray[cheapest[node]+2]
                    set1 = self.find(parent, u) 
                    set2 = self.find(parent ,v) 
  
                    if set1 != set2 : 
                        MSTweight += w
                        self.union(parent, rank, set1, set2)
#                         print("This is the Parent array: ",parent)
#                         print ("Edge %d-%d with weight %d included in MST" % (u,v,w)) 
                        numTrees = numTrees - 1
            #reset cheapest array 
            cheapest =[-1] * self.V
            cheapest2 = make_numpy(cheapest)
            cheapest_gpu.set(cheapest2)
            parent_gpu.set(parent)

        parent = parent_gpu.get()
#         cheapest_check = cheapest_gpu.get()
        print ("Weight of MST is %d" % MSTweight) 
#         print("This is the parent array: ", parent)

In [42]:
# # KNN
# distance = np.full(shape=(NUMBER_OF_NODES,NUMBER_OF_NODES),fill_value=float('inf'),dtype=np.float32)
# for i in GraphList:
#     distance[i[0],i[1]]=i[2]
    

In [43]:
# C = 2
# import math
# k = math.floor(math.log(NUMBER_OF_NODES)) + C

# # knnList
# for i in range(NUMBER_OF_NODES):
#     if(i%100==0):
#         print(i)
#     idx = np.argpartition(distance[i],k)
#     for j in range(NUMBER_OF_NODES):
#         if j not in idx[:k]:
#             distance[i,j]=float('inf')

In [44]:
FullGraphGPU = GraphParallel(NUMBER_OF_NODES)
for i in GraphList:
    FullGraphGPU.addEdge(i[0],i[1],i[2])
cArray,cHelper1,cHelper2 = FullGraphGPU.makeInputArray()

In [45]:
# FullGraphGPU = GraphParallel(NUMBER_OF_NODES)
# for i in range(len(distance)):
#     for j in range(i):
#         if distance[i,j]!=float('inf'):
#             FullGraphGPU.addEdge(i,j,distance[i,j])

In [46]:
time1 = time.clock()
MSTLinesGPU = FullGraphGPU.boruvkaMST(cArray,cHelper1,cHelper2)
time2 = time.clock()
print('GPU MST with full graph took %f seconds' % (time2-time1))
del FullGraphGPU

Number of Nodes:  200
Number of Edges:  1000
Size of block:  (1024, 1, 1)
Size of grid:  (1, 1)
Time CUDA code executes:  3.6319578384791384e-05
Time CUDA code executes:  2.6798912244885287e-05
Time CUDA code executes:  2.7151529479851888e-05
Weight of MST is 2770
This is the parent array:  [  4   5  15  45  64   4   4   4   4  64   3  21   7  19  82   4  15   7
  19   7   4  64   5   2   5  64   0   6   4   5  24  46  55  20  45   9
  20  30  46   1   9  15  17  11   4  64   4  17   4   4  64   9  49  15
  46  50  44   2  55  64   4  21  30   1  64  64  45  59  15  60  21   2
  15  35  60   5  24  28  50  34   5   1  64  35  15   2  50  68  15  64
  21  82  45  15   4  39  64   8  21  13  31  46  17  68   9   9  46  90
  15  65  64  24  80  22  17  90  66  94  50   4  25  55 119 118   0   5
   1  19  38 170  55  16  71  24  20  11  51  64  19  14  50  50   6  45
  59  64   9   0  85  60  21   4  31 118  94  32  15 124 151 145  68  45
  28  94  81  30  71  71 149 110  15  50  64 149   

In [ ]:
time1=time.clock()
testingGNM = nx.gnm_random_graph(5000000,50000000,seed=24,directed=False)
GraphList = []
for x,(i,j) in enumerate(testingGNM.edges()):
    GraphList.append([i,j,int(99*rnd())+1])
time2=time.clock()
print("Time to generate graph: ",time2-time1)

# -------------------------- Checking to see if it is connected and storing it
print(nx.is_connected(testingGNM))
del testingGNM